This code geenrates optimum frequencies for corral/fenceline modules given some constraints, such that all parametric conversions, gfby2 terms, and qubits/snail frequecnies are maximally separated.

In [1]:
import itertools
import numpy as np
import scipy.optimize as sc
from scipy.optimize import minimize, Bounds, shgo, differential_evolution


In [2]:
class freq_term():

    def __init__(self,freq,highthreshold,lowthreshold,info=""):
        self.freq = freq
        self.threshold = (lowthreshold,highthreshold)
        self.low_bound = freq-lowthreshold
        self.high_bound = freq+highthreshold
        self.info = info
        return

def generate_freq_classes(fq_dict, lowthresh, upthresh):
    freqs_classes = [freq_term(freq,upthresh,lowthresh,info) for (info,freq) in fq_dict.items() ]
    return freqs_classes

#### For the entire Corral module

In [ ]:
def generate_frequencies(q_fq,s_fq):
    def q_an_fq(f):
        return f - 0.17

    def q_gain_f(f1,f2):
        return (f1+f2)

    def q_conv_f(f1,f2):
        return np.abs(f1-f2)

    def combinations(array):
        return list(itertools.combinations(array,2))

    # all paramteric relations where only one snail is activated a time
    q_par_relations_1 = list(itertools.combinations([0,1,2,3],2)) # module 1
    q_par_relations_2 = list(itertools.combinations([2,3,4,5],2)) # module 2
    q_par_relations_3 = list(itertools.combinations([4,5,1,0],2)) # module 3

    # all qubit frequencies
    q_fqs = {f"q{k}":q_fq[k] for (k,i) in enumerate(q_fq)}

    # all snail frequencies
    s_fqs = {f"s{k}":s_fq[k] for (k,i) in enumerate(s_fq)}

    # all anharmonic frequencies
    q_an_fqs = {f"q{k}_an":q_an_fq(i) for (k,i) in enumerate(q_fq)}

    # all gf/2 frequencies
    q_gfby2_fqs = {f"q{k}_gfby2":q_an_fq(i)/2 for (k,i) in enumerate(q_fq)}

    #all gain frequencies one block at a time
    gain_fqs = {f"q{i[0]}_q{i[1]}_gain":q_gain_f(q_fq[i[0]],q_fq[i[1]]) for (k,i) in enumerate(q_par_relations_1)} |\
               {f"q{i[0]}_q{i[1]}_gain":q_gain_f(q_fq[i[0]],q_fq[i[1]]) for (k,i) in enumerate(q_par_relations_2)} |\
               {f"q{i[0]}_q{i[1]}_gain":q_gain_f(q_fq[i[0]],q_fq[i[1]]) for (k,i) in enumerate(q_par_relations_3)} # | unites the dicts, \ is for line continuity
    # all conversion frequencies one block at a time
    conv_fqs =  {f"q{i[0]}_q{i[1]}_conv":q_conv_f(q_fq[i[0]],q_fq[i[1]]) for (k,i) in enumerate(q_par_relations_1)} |\
                {f"q{i[0]}_q{i[1]}_conv":q_conv_f(q_fq[i[0]],q_fq[i[1]]) for (k,i) in enumerate(q_par_relations_2)} |\
                {f"q{i[0]}_q{i[1]}_conv":q_conv_f(q_fq[i[0]],q_fq[i[1]]) for (k,i) in enumerate(q_par_relations_3)}     # all parametric gate frequencies one bloack at a time

    # make a dictionary out of all calucalted frequencies
    # freqs = q_fqs | s_fqs | q_an_fqs | q_gfby2_fqs | gain_fqs | conv_fqs

    return [q_fqs , s_fqs , q_an_fqs , q_gfby2_fqs , gain_fqs , conv_fqs]

def np_objective_funct_v2(freqss):
    '''
    Implementation of a cost function based on 1/(x_1-x_2) potential function. Takes in a list of frequencies and evaluates a cost with boundary conditions.
    The weights can be provided in the arguments, but not for now.
    '''
    q_fqs,s_fqs = freqss[0:6], freqss[-2:]
    # print(q_fqs)
    # print(s_fqs)

    q_fqs , s_fqs , q_an_fqs , q_gfby2_fqs , gain_fqs , conv_fqs = generate_frequencies(q_fqs,s_fqs)
    #cost of gain-gain, gain-conversion, conversion conversion processes one block at a time
    try:

        # cost of qubit fqs with all others:
        cost_q_q = 5*sum([1/abs(i[0][1]-i[1][1]) for i in itertools.combinations(q_fqs.items(),2)])
        cost_q_s = 5*sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(s_fqs.items(),q_fqs.items())])
        cost_q_qan = 5*sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(q_an_fqs.items(),q_fqs.items())])
        cost_q_qgfby2 = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(q_gfby2_fqs.items(),q_fqs.items())])
        cost_q_gain = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(gain_fqs.items(),q_fqs.items())])
        cost_q_conv = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(conv_fqs.items(),q_fqs.items())])

        # cost of snail fqs with all others:
        cost_s_s = 5*sum([1/abs(i[0][1]-i[1][1]) for i in itertools.combinations(s_fqs.items(),2)])
        cost_s_q_an = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(s_fqs.items(),q_an_fqs.items())])
        cost_s_q_gfby2 = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(s_fqs.items(),q_gfby2_fqs.items())])
        cost_s_gain = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(s_fqs.items(),gain_fqs.items())])
        cost_s_conv = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(s_fqs.items(),conv_fqs.items())])

        #cost of qubit anharmonicites with others
        cost_qan_qan = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.combinations(q_an_fqs.items(),2)])
        cost_qan_gfby2 = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(q_an_fqs.items(),q_gfby2_fqs.items())])
        cost_qan_gain = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(q_an_fqs.items(),gain_fqs.items())])
        cost_qan_conv = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(q_an_fqs.items(),conv_fqs.items())])

        #cost of gain frequencies:
        cost_gain_gain = 5*sum([1/abs(i[0][1]-i[1][1]) for i in itertools.combinations(gain_fqs.items(),2)])
        cost_gain_conv = 5*sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(conv_fqs.items(),gain_fqs.items())])

        # cost of coversion:
        cost_conv_conv = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.combinations(conv_fqs.items(),2)])

    except ZeroDivisionError:return np.inf

    total_cost = cost_q_q+cost_q_s+cost_q_qan+cost_q_qgfby2+cost_q_gain+cost_q_conv+\
                cost_s_s+cost_s_q_an+cost_s_q_gfby2+cost_s_gain+cost_s_conv+\
                cost_qan_qan+cost_qan_gfby2+cost_qan_gain+cost_qan_conv+\
                cost_gain_gain+cost_gain_conv+cost_conv_conv
    return total_cost


Differential evolution and trust-constr:
(Both don't perform well)

In [ ]:
q_fq = [3.1,5.6,4.2,6.2,5.3,4.9]
s_fq = [7.5,6.9]
freqss = q_fq.append(s_fq)
bounds = ([[2.9,6], [2.9,6], [2.9,6], [2.9,6],[2.9,6],[2.9,6],[4,8],[4,8]])
results1 = sc.differential_evolution(np_objective_funct_v2, x0 =freqss, bounds=bounds,maxiter=1000)
print(results1)


q_fq = [3.1,5.6,4.2,6.2,5.3,4.9,7.5,6.9]
bounds = sc.Bounds([2.9,2.9,2.9,2.9,2.9,2.9,5,5],[6,6,6,6,6,6,8,8])
results1 = sc.minimize(np_objective_funct_v2, x0 =q_fq,method='trust-constr', bounds=bounds, options={'maxiter': 10000})
print(results1)

#### No boundary conditions - 'Fence-line'

We let go of wrapping around in the Corral architecture to ease some constraints.

In [ ]:
def generate_frequencies_noboundary(q_fq,s_fq):
    def q_ef_fq(f):
        return f - 0.17

    def q_gfby2_fq(f):
        return f - 0.17/2

    def q_gain_f(f1,f2):
        return (f1+f2)

    def q_conv_f(f1,f2):
        return np.abs(f1-f2)

    # all paramteric relations where only one snail is activated a time
    q_par_relations_1 = list(itertools.combinations([0,1,2,3],2)) # module 1
    q_par_relations_2 = list(itertools.combinations([2,3,4,5],2)) # module 2
    # q_par_relations_3 = list(itertools.combinations([4,5,1,0],2)) # module 3

    # all qubit frequencies
    q_fqs = {f"q{k}":q_fq[k] for (k,i) in enumerate(q_fq)}

    # all snail frequencies
    s_fqs = {f"s{k}":s_fq[k] for (k,i) in enumerate(s_fq)}

    # all gf/2 frequencies
    q_gfby2_fqs = {f"q{k}_gfby2":q_gfby2_fq(i) for (k,i) in enumerate(q_fq)}

    #all gain frequencies one block at a time
    gain_fqs = {f"q{i[0]}_q{i[1]}_gain":q_gain_f(q_fq[i[0]],q_fq[i[1]]) for (k,i) in enumerate(q_par_relations_1)} |\
               {f"q{i[0]}_q{i[1]}_gain":q_gain_f(q_fq[i[0]],q_fq[i[1]]) for (k,i) in enumerate(q_par_relations_2)}
    # all conversion frequencies one block at a time
    conv_fqs =  {f"q{i[0]}_q{i[1]}_conv":q_conv_f(q_fq[i[0]],q_fq[i[1]]) for (k,i) in enumerate(q_par_relations_1)} |\
                {f"q{i[0]}_q{i[1]}_conv":q_conv_f(q_fq[i[0]],q_fq[i[1]]) for (k,i) in enumerate(q_par_relations_2)}

    return [q_fqs , s_fqs , q_gfby2_fqs , gain_fqs , conv_fqs ]


def generate_frequencies_noboundary_v2(q_fq,s_fq):
    '''An extended version of the above that includes way more terms, that may or may not matter.'''
    def q_ef_fq(f):
        return f - 0.17

    def q_gfby2_fq(f):
        return f - 0.17/2

    def q_gain_f(f1,f2):
        return (f1+f2)

    def q_conv_f(f1,f2):
        return np.abs(f1-f2)

    def ef_to_fe(f1,f2):
        ef = f1 + 2* f2 - 0.17
        fe = 2*f1 - 0.17 + f2
        return np.abs(ef-fe)

    def ee_to_fg(f1,f2):
        ee = f1+f2
        fg = 2*f1 - 0.17
        return np.abs(ee-fg)

    def ee_to_gf(f1,f2):
        ee = f1+f2
        gf = 2*f2 - 0.17
        return np.abs(ee-gf)


    # all paramteric relations where only one snail is activated a time
    q_par_relations_1 = list(itertools.combinations([0,1,2,3],2)) # module 1
    q_par_relations_2 = list(itertools.combinations([2,3,4,5],2)) # module 2
    # q_par_relations_3 = list(itertools.combinations([4,5,1,0],2)) # module 3

    # all qubit frequencies
    q_fqs = {f"q{k}":q_fq[k] for (k,i) in enumerate(q_fq)}

    # subharmonic qubit frequencies
    q_sub_fqs = {f"q_sub_{k}":q_fq[k]/3 for (k,i) in enumerate(q_fq)}

    # all snail frequencies
    s_fqs = {f"s{k}":s_fq[k] for (k,i) in enumerate(s_fq)}

    # all ef frequencies
    q_ef_fqs = {f"q{k}_ef":q_ef_fq(i) for (k,i) in enumerate(q_fq)}

    # all gf/2 frequencies
    q_gfby2_fqs = {f"q{k}_gfby2":q_gfby2_fq(i) for (k,i) in enumerate(q_fq)}

    # snail qubit subharmonic frequencies
    q_s_sub_fqs = {f"q{k}_s_sub":q_fq[k]/2 for (k,i) in enumerate(q_fq)}

    # for |ef> to |fe>
    q_ef_to_fe_fqs = {f"q{i[0]}_q{i[1]}_ef_to_fe":ef_to_fe(q_fq[i[0]],q_fq[i[1]]) for (k,i) in enumerate(q_par_relations_1)}|\
                     {f"q{i[0]}_q{i[1]}_ef_to_fe":ef_to_fe(q_fq[i[0]],q_fq[i[1]]) for (k,i) in enumerate(q_par_relations_2)}

    # for |ee> to |fg>
    q_ee_to_fg_fqs = {f"q{i[0]}_q{i[1]}_ee_to_fg":ee_to_fg(q_fq[i[0]],q_fq[i[1]]) for (k,i) in enumerate(q_par_relations_1)}|\
                     {f"q{i[0]}_q{i[1]}_ee_to_fg":ee_to_fg(q_fq[i[0]],q_fq[i[1]]) for (k,i) in enumerate(q_par_relations_2)}

    # for |ee> to |gf>
    q_ee_to_gf_fqs = {f"q{i[0]}_q{i[1]}_ee_to_gf":ee_to_gf(q_fq[i[0]],q_fq[i[1]]) for (k,i) in enumerate(q_par_relations_1)}|\
                     {f"q{i[0]}_q{i[1]}_ee_to_gf":ee_to_gf(q_fq[i[0]],q_fq[i[1]]) for (k,i) in enumerate(q_par_relations_2)}

    #all gain frequencies one block at a time
    gain_fqs = {f"q{i[0]}_q{i[1]}_gain":q_gain_f(q_fq[i[0]],q_fq[i[1]]) for (k,i) in enumerate(q_par_relations_1)} |\
               {f"q{i[0]}_q{i[1]}_gain":q_gain_f(q_fq[i[0]],q_fq[i[1]]) for (k,i) in enumerate(q_par_relations_2)} #|\
            #    {f"q{i[0]}_q{i[1]}_gain":q_gain_f(q_fq[i[0]],q_fq[i[1]]) for (k,i) in enumerate(q_par_relations_3)} # | unites the dicts, \ is for line continuity
    # all conversion frequencies one block at a time
    conv_fqs =  {f"q{i[0]}_q{i[1]}_conv":q_conv_f(q_fq[i[0]],q_fq[i[1]]) for (k,i) in enumerate(q_par_relations_1)} |\
                {f"q{i[0]}_q{i[1]}_conv":q_conv_f(q_fq[i[0]],q_fq[i[1]]) for (k,i) in enumerate(q_par_relations_2)} #|\
                # {f"q{i[0]}_q{i[1]}_conv":q_conv_f(q_fq[i[0]],q_fq[i[1]]) for (k,i) in enumerate(q_par_relations_3)}     # all parametric gate frequencies one bloack at a time

    # make a dictionary out of all calucalted frequencies
    # freqs = q_fqs | s_fqs | q_an_fqs | q_gfby2_fqs | gain_fqs | conv_fqs

    return [q_fqs , s_fqs , q_gfby2_fqs , gain_fqs , conv_fqs, q_sub_fqs, q_s_sub_fqs, q_ef_fqs, q_ef_to_fe_fqs, q_ee_to_fg_fqs, q_ee_to_gf_fqs ]


def check_collisions(freqss):
    '''Assign boundaries to frequencies and cehck for collisions'''
    freq_terms_list = generate_frequencies_noboundary(freqss[:6],freqss[6:]) # best one

    # [4.004,5.062,4.168,4.549],[4.275] # MK's SM freqs


    all_freq_classes = []

    # qubit terms
    for term in freq_terms_list[0].items():
        all_freq_classes.append(freq_term(term[1],0.17,0.17,term[0]))

    # snail terms
    for term in freq_terms_list[1].items():
        all_freq_classes.append(freq_term(term[1],0.08,0.02,term[0]))

    # gfby2 terms
    for term in freq_terms_list[2].items():
        all_freq_classes.append(freq_term(term[1],0.17,0,term[0]))

    # gain terms
    for term in freq_terms_list[3].items():
        all_freq_classes.append(freq_term(term[1],0.05,0.05,term[0]))

    # conv terms
    for term in freq_terms_list[4].items():
        all_freq_classes.append(freq_term(term[1],0.1,0.1,term[0]))

    # implement combinations
    all_freqs_combs = list(itertools.combinations(all_freq_classes,2))


    # check for collisions
    for (t1,t2) in all_freqs_combs:
        if (t1.low_bound <= t2.low_bound and t1.high_bound >= t2.low_bound) or (t2.low_bound <= t1.low_bound and t2.high_bound >= t1.low_bound):
            print(f"{t1.info} {t1.freq:.3f} GHz, {t2.info} {t2.freq:.3f} GHz, DIFF : {t1.freq-t2.freq:.3f} GHz")
    return


In [ ]:
# random trial to see the frquencies generated
freq_terms_list = generate_frequencies_noboundary([3.198e+00,  5.752e+00,  3.789e+00,  6.000e+00,  5.216e+00, 4.791e+00],  [6.892e+00,  6.527e+00]) # best one

all_freq_dict = freq_terms_list[0] | freq_terms_list[1] | freq_terms_list[2] | freq_terms_list[3] | freq_terms_list[4] | freq_terms_list[5] | freq_terms_list[6]

all_freq_dict = freq_terms_list[2]


sorted_list = sorted(all_freq_dict.items(), key=lambda x: x[1])

for i in sorted_list:
    print(i[0], i[1])

##### Optimisation using Nelder-Mead

In [ ]:
def np_objective_funct_optim(freqss):
    '''
    Implementation of a cost function based on 1/(x_1-x_2) potential function. Terms are explicitly defined to see the effect of weights.
    '''
    q_fqs,s_fqs = freqss[0:6], freqss[-2:]

    q_fqs , s_fqs , q_gfby2_fqs , gain_fqs , conv_fqs = generate_frequencies_noboundary(q_fqs,s_fqs)
    #cost of gain-gain, gain-conversion, conversion conversion processes one block at a time
    try:

        # cost of qubit fqs with all others:
        cost_q_q = 10*sum([1/abs(i[0][1]-i[1][1]) for i in itertools.combinations(q_fqs.items(),2)])
        cost_q_s = 10*sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(s_fqs.items(),q_fqs.items())])
        # cost_q_qan = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(q_an_fqs.items(),q_fqs.items())])
        cost_q_qgfby2 = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(q_gfby2_fqs.items(),q_fqs.items())])
        cost_q_gain = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(gain_fqs.items(),q_fqs.items())])
        cost_q_conv = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(conv_fqs.items(),q_fqs.items())])

        # # cost of snail fqs with all others:
        cost_s_s = 10*sum([1/abs(i[0][1]-i[1][1]) for i in itertools.combinations(s_fqs.items(),2)])
        # cost_s_q_an = 4*sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(s_fqs.items(),q_an_fqs.items())])
        cost_s_q_gfby2 = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(s_fqs.items(),q_gfby2_fqs.items())])
        cost_s_gain = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(s_fqs.items(),gain_fqs.items())])
        cost_s_conv = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(s_fqs.items(),conv_fqs.items())])

        # #cost of qubit anharmonicites with others
        # cost_qan_qan = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.combinations(q_an_fqs.items(),2)])
        # cost_qan_gfby2 = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(q_an_fqs.items(),q_gfby2_fqs.items())])
        # cost_qan_gain = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(q_an_fqs.items(),gain_fqs.items())])
        # cost_qan_conv = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(q_an_fqs.items(),conv_fqs.items())])

        # #cost of gain frequencies:
        # cost_gain_gain = 10*sum([1/abs(i[0][1]-i[1][1]) for i in itertools.combinations(gain_fqs.items(),2)])
        # cost_gain_conv = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(conv_fqs.items(),gain_fqs.items())])

        # # cost of coversion:
        cost_conv_conv = 10*sum([1/abs(i[0][1]-i[1][1]) for i in itertools.combinations(conv_fqs.items(),2)])

    except ZeroDivisionError:return 1e14

    total_cost = cost_q_q+cost_q_s+cost_q_qgfby2+cost_q_gain+cost_q_conv+\
                cost_s_s+cost_s_q_gfby2+cost_s_gain+cost_s_conv+cost_conv_conv
                # cost_gain_gain+cost_gain_conv#+
    return total_cost

In [ ]:
bounds = Bounds([2.9,2.9,2.9,2.9,2.9,2.9,3,5.2],[6,6,6,6,6,6,5.5,5.4])

x0 = [3.1,5.6,4.2,6.2,5.3,4.9,4.3,5.3]
results1 = minimize(np_objective_funct_optim, x0 =x0,method='nelder-mead', bounds=bounds, options={'disp': True,'maxiter': 10000})
results1


In [ ]:
check_collisions(results1['x'])

trying out SLSQP and trust-constr given an 'okay' guess. Global optimzation does not converge.

In [ ]:
bounds = Bounds([2.9,2.9,2.9,2.9,2.9,2.9,3,5.2],[6,6,6,6,6,6,5.5,5.4])

# x0 = [3.156e+00,  5.748e+00,  4.088e+00,  6.000e+00,  5.367e+00, 4.889e+00,  8.000e+00,  7.019e+00]
# results2 = shgo(np_objective_funct_optim, bounds=bounds, options={'disp': True,'maxiter': 1000, 'maxtime' : 10}, workers=-1)
# results3 = differential_evolution(np_objective_funct_optim, bounds=bounds, options={'disp': True,'maxiter': 1000})

x0 = [3.035e+00,  5.781e+00,  4.005e+00,  6.000e+00,  5.505e+00, 4.828e+00,  4.369e+00,  5.200e+00]
results2 = minimize(np_objective_funct_optim, x0 =x0,method='SLSQP', bounds=bounds, options={'disp': True,'maxiter': 1000})
print(results2)

x0 = [3.035e+00,  5.781e+00,  4.005e+00,  6.000e+00,  5.505e+00, 4.828e+00,  4.369e+00,  5.200e+00]
results3 = minimize(np_objective_funct_optim, x0 =x0,method='trust-constr', bounds=bounds, options={'disp': True,'maxiter': 1000})
print(results3)

bounds = Bounds([2.9,2.9,2.9,2.9,2.9,2.9,5,5],[6,6,6,6,6,6,8,8])


#### Single Module 

Using the result that fidelities don't get affected a lot due to repeating a module's frequencies, we try out Nelder-Mead for one single module.

In [12]:
def generate_frequencies_noboundary_sm(q_fq,s_fq):
    def q_an_fq(f):
        return f - 0.17

    def q_gain_f(f1,f2):
        return (f1+f2)

    def q_conv_f(f1,f2):
        return np.abs(f1-f2)

    # all paramteric relations where only one snail is activated a time
    q_par_relations_1 = list(itertools.combinations([0,1,2,3],2)) # module 1

    # all qubit frequencies
    q_fqs = {f"q{k}":q_fq[k] for (k,i) in enumerate(q_fq)}
    # all snail frequencies
    s_fqs = {f"s{k}":s_fq[k] for (k,i) in enumerate(s_fq)}
    # all gf/2 frequencies
    q_gfby2_fqs = {f"q{k}_gfby2":q_fq[k]-0.17/2 for (k,i) in enumerate(q_fq)}
    #all gain frequencies
    gain_fqs = {f"q{i[0]}_q{i[1]}_gain":q_gain_f(q_fq[i[0]],q_fq[i[1]]) for (k,i) in enumerate(q_par_relations_1)}
   # all conversion frequencies
    conv_fqs =  {f"q{i[0]}_q{i[1]}_conv":q_conv_f(q_fq[i[0]],q_fq[i[1]]) for (k,i) in enumerate(q_par_relations_1)}

    return [q_fqs , s_fqs , q_gfby2_fqs , gain_fqs , conv_fqs]


def np_objective_funct_optim(freqss):
    '''
    Implementation of a cost function based on 1/(x_1-x_2) potential function.
    '''
    q_fqs,s_fqs = freqss[0:4], [freqss[-1]]

    q_fqs , s_fqs , q_gfby2_fqs , gain_fqs , conv_fqs = generate_frequencies_noboundary_sm(q_fqs,s_fqs)
    #cost of gain-gain, gain-conversion, conversion conversion processes one block at a time
    try:

        # cost of qubit fqs with all others:
        cost_q_q = 10*sum([1/abs(i[0][1]-i[1][1]) for i in itertools.combinations(q_fqs.items(),2)])
        cost_q_s = 10*sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(s_fqs.items(),q_fqs.items())])
        cost_q_qgfby2 = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(q_gfby2_fqs.items(),q_fqs.items())])
        cost_q_gain = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(gain_fqs.items(),q_fqs.items())])
        cost_q_conv = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(conv_fqs.items(),q_fqs.items())])

        # # cost of snail fqs with all others:
        cost_s_s = 10*sum([1/abs(i[0][1]-i[1][1]) for i in itertools.combinations(s_fqs.items(),2)])
        cost_s_q_gfby2 = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(s_fqs.items(),q_gfby2_fqs.items())])
        cost_s_gain = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(s_fqs.items(),gain_fqs.items())])
        cost_s_conv = sum([1/abs(i[0][1]-i[1][1]) for i in itertools.product(s_fqs.items(),conv_fqs.items())])

        # # cost of coversion:
        cost_conv_conv = 10*sum([1/abs(i[0][1]-i[1][1]) for i in itertools.combinations(conv_fqs.items(),2)])

    except ZeroDivisionError:return 1e14

    total_cost = cost_q_q+cost_q_s+cost_q_qgfby2+cost_q_gain+cost_q_conv+\
                cost_s_s+cost_s_q_gfby2+cost_s_gain+cost_s_conv+cost_conv_conv
    return total_cost

def check_collisions(freqss):
    '''
    Check if (f0_min ---- f0 ----- f0_max) collides with (f1_min ---- f1 ----- f1_max).
    Assuming min and max is the 'fab' precision.
    '''
    freq_terms_list = generate_frequencies_noboundary_sm(freqss[:4],[freqss[-1]]) # best one

    # [4.004,5.062,4.168,4.549],[4.275] # MK's SM freqs

    all_freq_classes = []

    # qubit terms, taking fab precision to be wildly variant
    for term in freq_terms_list[0].items():
        all_freq_classes.append(freq_term(term[1],0.5,0.5,term[0]))

    # snail terms
    for term in freq_terms_list[1].items():
        all_freq_classes.append(freq_term(term[1],0.5,0.5,term[0]))

    # gfby2 terms
    for term in freq_terms_list[2].items():
        all_freq_classes.append(freq_term(term[1],0.17,0,term[0]))

    # the boundaries are 0.25 here as \Delta f_param = (\Delta f_0 + \Delta f_1)/2
    # gain terms
    for term in freq_terms_list[3].items():
        all_freq_classes.append(freq_term(term[1],0.25,0.25,term[0]))

    # conv terms
    for term in freq_terms_list[4].items():
        all_freq_classes.append(freq_term(term[1],0.25,0.25,term[0]))

    # implement combinations
    all_freqs_combs = list(itertools.combinations(all_freq_classes,2))


    # check for collisions
    for (t1,t2) in all_freqs_combs:
        if (t1.low_bound <= t2.low_bound and t1.high_bound >= t2.low_bound) or (t2.low_bound <= t1.low_bound and t2.high_bound >= t1.low_bound):
            print(f"{t1.info} {t1.freq:.3f} GHz, {t2.info} {t2.freq:.3f} GHz, DIFF : {t1.freq-t2.freq:.3f} GHz")
    return

In [10]:
bounds = Bounds([3.3,3.3,3.3,3.3,4.2],[5.7,5.7,5.7,5.7,4.7])

x0 = [3.1,5.6,4.2,6.2,4.5]
results1 = minimize(np_objective_funct_optim, x0 =x0,method='nelder-mead', bounds=bounds, options={'disp': True,'maxiter': 10000})
results1

Optimization terminated successfully.
         Current function value: 410.460676
         Iterations: 108
         Function evaluations: 188


C:\Users\Hatlab-Chao\AppData\Local\Temp\ipykernel_19220\851332897.py:4: OptimizeWarning: Initial guess is not within the specified bounds
  results1 = minimize(np_objective_funct_optim, x0 =x0,method='nelder-mead', bounds=bounds, options={'disp': True,'maxiter': 10000})
C:\Users\Hatlab-Chao\AppData\Local\Temp\ipykernel_19220\2985405547.py:55: RuntimeWarning: divide by zero encountered in scalar divide
  cost_q_q = 10*sum([1/abs(i[0][1]-i[1][1]) for i in itertools.combinations(q_fqs.items(),2)])
C:\Users\Hatlab-Chao\AppData\Local\Temp\ipykernel_19220\2985405547.py:80: RuntimeWarning: divide by zero encountered in scalar divide
  cost_conv_conv = 10*sum([1/abs(i[0][1]-i[1][1]) for i in itertools.combinations(conv_fqs.items(),2)])


       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 410.4606758124895
             x: [ 3.300e+00  5.332e+00  4.075e+00  5.700e+00  4.671e+00]
           nit: 108
          nfev: 188
 final_simplex: (array([[ 3.300e+00,  5.332e+00, ...,  5.700e+00,
                         4.671e+00],
                       [ 3.300e+00,  5.332e+00, ...,  5.700e+00,
                         4.671e+00],
                       ...,
                       [ 3.300e+00,  5.332e+00, ...,  5.700e+00,
                         4.671e+00],
                       [ 3.300e+00,  5.332e+00, ...,  5.700e+00,
                         4.671e+00]]), array([ 4.105e+02,  4.105e+02,  4.105e+02,  4.105e+02,
                        4.105e+02,  4.105e+02]))

In [11]:
check_collisions(results1.x)

q0 3.300 GHz, q2 4.075 GHz, DIFF : -0.775 GHz
q0 3.300 GHz, q0_gfby2 3.215 GHz, DIFF : 0.085 GHz
q1 5.332 GHz, q3 5.700 GHz, DIFF : -0.368 GHz
q1 5.332 GHz, s0 4.671 GHz, DIFF : 0.662 GHz
q1 5.332 GHz, q1_gfby2 5.247 GHz, DIFF : 0.085 GHz
q1 5.332 GHz, q3_gfby2 5.615 GHz, DIFF : -0.283 GHz
q2 4.075 GHz, s0 4.671 GHz, DIFF : -0.596 GHz
q2 4.075 GHz, q2_gfby2 3.990 GHz, DIFF : 0.085 GHz
q3 5.700 GHz, q1_gfby2 5.247 GHz, DIFF : 0.453 GHz
q3 5.700 GHz, q3_gfby2 5.615 GHz, DIFF : 0.085 GHz
q0_q1_gain 8.632 GHz, q0_q3_gain 9.000 GHz, DIFF : -0.368 GHz
q0_q3_gain 9.000 GHz, q1_q2_gain 9.407 GHz, DIFF : -0.407 GHz
q1_q2_gain 9.407 GHz, q2_q3_gain 9.775 GHz, DIFF : -0.368 GHz
q0_q1_conv 2.032 GHz, q0_q3_conv 2.400 GHz, DIFF : -0.368 GHz
q0_q1_conv 2.032 GHz, q2_q3_conv 1.625 GHz, DIFF : 0.407 GHz
q0_q2_conv 0.775 GHz, q1_q2_conv 1.258 GHz, DIFF : -0.483 GHz
q0_q2_conv 0.775 GHz, q1_q3_conv 0.368 GHz, DIFF : 0.407 GHz
q1_q2_conv 1.258 GHz, q2_q3_conv 1.625 GHz, DIFF : -0.368 GHz


A in-depth analysis with `generate_frequencies_noboundary_v2` would be beneficial to see what other terms can interfere here. For now, that is not the focus.